## Initial exploratory analysis

In [3]:
## importing libraries
import pandas as pd
import numpy as np
import geopy.distance 
from dateutil.relativedelta import relativedelta
from datetime import datetime
import os

## Here we set the directory to look up the folder containing the data
os.chdir("/Users/gabrielmedeiros/Documents/OneDrive/Business analytics/DATA_445_Machine_Learning")


## Reading csv file, index_col = 0 makes the first column of the data to become the index of our pandas data frame
test_data = pd.read_csv('fraudTest.csv', index_col = 0)

train_data = pd.read_csv('fraudTrain.csv', index_col = 0)

n = test_data.shape[0]

/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


## Head of Train data

In [ ]:
print(train_data.shape)
train_data.head()

## Head of Test Data

In [ ]:
print(test_data.shape)
test_data.head()

## Feature Engineering

In [ ]:

##############
## Distance ##
##############


## Here assign to n the amount of observations we have in the dataset
n = test_data.shape[0]

## Creating empty list to store the reuslts
distance_to_append = []

## Looping through each row an computing the distance between ?????????
for i in range(0,n):
    
    ## Here we gather the lat and long from the ?????
    coords_1 = (test_data['lat'][i], test_data['long'][i])
    
    ## Here we gather the lat and long from the ?????
    coords_2 = (test_data['merch_lat'][i], test_data['merch_long'][i])
    
    ## Here we compute the disance in miles between the locations
    distance_to_append.append(geopy.distance.geodesic(coords_1, coords_2).miles)

## Adding results to our data set
test_data['Distance'] = distance_to_append


##################
## AVG Distance ##
##################

## Here we create a groupby function to get the mean distance by each category
avg_dist_by_category = pd.DataFrame(test_data.groupby(['cc_num','category'])['Distance'].mean())

## Here we create a temporary column holding our index values (cc_num)
avg_dist_by_category['columns'] = avg_dist_by_category.index

## Here we drop our temporary column and reset our index, which was previously our cc_num
avg_dist_by_category = avg_dist_by_category.reset_index().drop(columns = 'columns')

## Here we rename the columns of the groupby function
avg_dist_by_category.columns = ['cc_num','category','avg_distance_by_category']

## Here we merge our temporary data frame with our data set
test_data = avg_dist_by_category.merge(test_data, on = ['cc_num','category'], how = 'left')


#############################
## AVG Distance by Category##
#############################

## Here we create a groupby function to get the mean distance by each category
avg_dist = pd.DataFrame(test_data.groupby(['cc_num'])['Distance'].mean())

## Here we create a temporary column holding our index values (cc_num)
avg_dist['columns'] = avg_dist.index

## Here we drop our temporary column and reset our index, which was previously our cc_num
avg_dist = avg_dist.reset_index().drop(columns = 'columns')

## Here we rename the columns of the groupby function
avg_dist.columns = ['cc_num','avg_distance']

## Here we merge our temporary data frame with our data set
test_data = avg_dist.merge(test_data, on = ['cc_num'], how = 'left')

#########
## Age ##
#########

## Creating empty list to store the resutls
ages_to_append = []

## Looping through each observation and computing the age of each individual from its DOB
for i in range(0,n):
    
    ## Here we add the last date of this year
    year_of_2021 = datetime.strptime('2021-12-31', "%Y-%m-%d")
    
    ## Here we call each DOB in the dataset
    dob = datetime.strptime(test_data.dob[i], "%Y-%m-%d")
    
    ## Here we compute the ages
    ages_to_append.append(relativedelta(year_of_2021, dob).years)

## Adding results to our data set
test_data['Age'] = ages_to_append

######################
## Days of the Week ##
######################

## Here we change the format of our date column
dates = pd.to_datetime(test_data['trans_date_trans_time'])

## Here we use our library to get the day of the week based on the transformed column
test_data['DayOfWeek'] = dates.dt.day_name()

#######################
## Month of the year ##
#######################

## Here we change the format of our date column
months = pd.to_datetime(test_data['trans_date_trans_time'])

## Here we use our library to get the month of the year based on the transformed column
test_data['Month'] = dates.dt.month

#####################
## Hour of the day ##
#####################

## Here we change the format of our date column
hour_of_the_day = []

## Here we loop throough each observation, change the format of the items in each column, 
## and retrieve the hour of the day
for i in range(0,n):
    hour_of_the_day.append(datetime.strptime(test_data.trans_date_trans_time[i] ,"%Y-%m-%d %H:%M:%S").hour)
    
## Here we attribute our results to a column
test_data['HourOfTheDay'] = hour_of_the_day

#################################
## Total card uses by customer ##
#################################

## Here we get the number of transactions shown in the data set per card
Uses = pd.DataFrame(test_data['cc_num'].value_counts())

## Here we create a column to keep our cc_num
Uses['cc_number2'] = Uses.index

## Here we reset our index
Uses = Uses.reset_index(drop = True)

## Here we rename our columns
Uses.columns = ['TotalUses','cc_num']


## Here we merge our temporary data frame with our data set
test_data = Uses.merge(test_data, on = 'cc_num', how = 'left')

####################################################
## Difference in minutes between each transaction ##
####################################################

## Here we create a list to hold our results
new_time = []

## Here we loop through each observation and transform the format of our data
for i in range(0,n):
    new_time.append(datetime.strptime(test_data.trans_date_trans_time[i] ,"%Y-%m-%d %H:%M:%S"))
    
## Here we create a column containing transformed data to compute the difference in minutes
## New format was recquired for the library to work
test_data['transformed_time'] = new_time

## Here we compute the difference in minutes between each transacion
test_data['DiffByCardTrans'] = test_data.groupby('cc_num')\
                               ['transformed_time'].diff().apply(lambda x: \
                               x/np.timedelta64(1, 'm')).fillna(0).astype('int64')


In [4]:
## Here we create a list to hold our results
new_time = []

n = test_data.shape[0]
## Here we loop through each observation and transform the format of our data
for i in range(0,n):
    new_time.append(datetime.strptime(test_data.trans_date_trans_time[i] ,"%Y-%m-%d %H:%M:%S"))
    
## Here we create a column containing transformed data to compute the difference in minutes
## New format was recquired for the library to work
test_data['transformed_time'] = new_time

## Difference in time by minutes for each card in the dataset

## Average amount

In [ ]:
amt_per_category = pd.DataFrame(test_data.groupby(['cc_num','category'])['amt'].mean())

amt_per_category['columns'] = amt_per_category.index

amt_per_category = amt_per_category.reset_index().drop(columns = 'columns')

amt_per_category.columns = ['cc_num','category','avg_by_category']

test_data = amt_per_category.merge(test_data, on = ['cc_num','category'], how = 'left')

In [ ]:
avg_amt = pd.DataFrame(test_data.groupby('cc_num')['amt'].mean())

avg_amt['columns'] = avg_amt.index

avg_amt = avg_amt.reset_index().drop(columns = 'columns')

avg_amt.columns = ['cc_num', 'avg_amt']

test_data = avg_amt.merge(test_data, on = 'cc_num', how = 'left')

In [ ]:
test_data.columns

In [ ]:
test_data['purchase_>_avg'] = np.where(test_data['amt'] > test_data['avg_amt'],1,0)

In [ ]:
test_data['purchase_>_avg_by_category'] = np.where(test_data['amt'] > test_data['avg_by_category'],1,0)

In [ ]:
test_data['purchase_>_distance'] = np.where(test_data['Distance'] > test_data['avg_distance'],1,0)

In [ ]:
test_data['purchase_>_distance_by_category'] = np.where(test_data['Distance'] > test_data['avg_distance_by_category'],1,0)

In [ ]:
temp_cc_time_max = pd.DataFrame(test_data.groupby('cc_num')['trans_date_trans_time'].max())
temp_cc_time_min = pd.DataFrame(test_data.groupby('cc_num')['trans_date_trans_time'].min())



temp_cc_time_max['columns'] = temp_cc_time_max.index
temp_cc_time_min['columns'] = temp_cc_time_min.index

temp_cc_time_max = temp_cc_time_max.reset_index().drop(columns = 'columns')
temp_cc_time_min = temp_cc_time_min.reset_index().drop(columns = 'columns')


temp_cc_time_max.columns = ['cc_num','max_date']
temp_cc_time_min.columns = ['cc_num','min_date']


test_data = temp_cc_time_max.merge(test_data, on = 'cc_num', how = 'left')
test_data = temp_cc_time_min.merge(test_data, on = 'cc_num', how = 'left')

In [ ]:
min_dates = pd.to_datetime(test_data.min_date) 
max_dates = pd.to_datetime(test_data.max_date)

diff_in_time = max_dates - min_dates
 
temp_data = pd.DataFrame(diff_in_time)
 
test_data['diff_first_last'] = temp_data.apply(lambda x: x/np.timedelta64(1, 'm')).fillna(0).astype('int64')

In [ ]:
test_data['diff_first_last'].describe()

In [ ]:
## Here we export the file containing all of the new feature to a csv file. 
#test_data.to_csv ("/Users/gabrielmedeiros/Documents/OneDrive/Business analytics/DATA_445_Machine_Learning/FraudTestData.csv", index = None, header=True)